In [6]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.dataset import TokenizedEditDataset
from coeditor.model import *
from coeditor.encoding import TokenizedEdit, decode_tokens, tokens_to_change
import shutil
import random
from prepare_data import make_or_load_datasets

os.chdir(proj_root())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# test_data_name = "medium"
test_data_name = "SPOT"
datasets = make_or_load_datasets(test_data_name, recreate_data=True)
test_data = datasets["test"]

Create tokenized edits: 100%|██████████| 4/4 [03:40<00:00, 55.07s/chunk] 


Tokenized dataset saved to: /mnt/nas/jiayi/coeditor/datasets/SPOT/AnalysisBasedEditEncoder(extra_ctx_names=('usees', 'post-usees'))
Dataset stats:
==================== test ====================
n_projects: 1
n_edits: 1375
input_size:
   mean: 2888.5
   median: 3029
   min: 70
   max: 4000
output_size:
   mean: 108.52
   median: 71
   min: 2
   max: 1704


In [7]:
rep_edits = pfilter(TokenizedEdit.is_repetitive_edit, test_data.all_edits())
small_edits = pfilter(TokenizedEdit.is_small_edit, test_data.all_edits())
print("Total edits: ", len(list(test_data.all_edits())))
print("Repetitive edits: ", len(rep_edits))
print("Small edits: ", len(small_edits))

filtering: 100%|██████████| 1375/1375 [00:00<00:00, 4632.32it/s]


Total edits:  1375
Repetitive edits:  72
Small edits:  794


In [8]:
# model_dir = get_model_dir(trained=True) / "coeditor-medium-file"
model_dir = get_model_dir(trained=True) / "coeditor-medium-analysis-post_usees"
model = CoeditorModel.load_pretrained(model_dir)
model.to("cuda:1")

eval_args = EvalArgs(4096 * 2)


In [9]:
rep_data = TokenizedEditDataset.from_edits(rep_edits)
dec_args = DecodingArgs(num_beams=1)
rep_result = model.predict_on_data(rep_data, eval_args, dec_args)
display(rep_result.exact_match_accuracy())

decoding: 100%|██████████| 31/31 [00:50<00:00,  1.64s/batch]


(mean=0.63889, weight=72)

In [5]:
rep_result.save_examples_to_dir(Path("output") / f"repetitive{model_dir.name}.txt", range(len(rep_result.edits)))

saving examples: 100%|██████████| 73/73 [00:02<00:00, 33.98it/s]


decoding: 100%|██████████| 36/36 [00:59<00:00,  1.66s/batch]


(mean=0.77011, weight=87)

In [4]:
small_data = TokenizedEditDataset.from_edits(small_edits)
dec_args = DecodingArgs(num_beams=1)
dec_result = model.predict_on_data(small_data, eval_args, dec_args)
display(dec_result.exact_match_accuracy())


decoding: 100%|██████████| 285/285 [06:50<00:00,  1.44s/batch]


(mean=0.18264, weight=772)

In [5]:
max_saved_samples = 200
random.seed(42)
exs_to_save = list(range(len(dec_result.predictions)))
random.shuffle(exs_to_save)
out_dir = model_dir / "predictions" / "small_edits"
dec_result.save_examples_to_dir(out_dir, exs_to_save[:max_saved_samples])

saving examples: 100%|██████████| 200/200 [00:02<00:00, 73.35it/s]
